# Communicate with your digital twin
This is a KWH HMI which allows to communicate with the digital twin via S³I-B protocol and HTTP REST. All necessary modules are imported into the script, including the S³I library and ml40 reference implementation. This Notebook acts as an HMI in terms of the S³I and a client in terms of OAuth2 authentication.
Just go to the Cell drop-down menu and click on the Run All button.

In [ ]:
import s3i
import json
import getpass
import uuid
from tools import print_with_timestamp, yes, no
from ml.tools import find_broker_endpoint, make_thing_config, load_config, make_feature_config
from ml.dt_factory import create_dt_ref, build_feature
from ml.app_logger import setup_logger

## Configure the notebook
In order to assign this notebook to your private HMI it needs a client id and the respective secret to authenticate itself. In the following input fields, you can enter the id and the secret of your HMI.

In [ ]:
print_with_timestamp("Assign a client to this notebook. (The id of your HMI)")
hmi_id = input('[S3I]: Please enter your HMI id:').strip('," ')
hmi_secret = getpass.getpass('[S3I]: Please enter your HMI secret:').strip('," ')
print_with_timestamp("HMI id and secret are set.")

Next, you have to enter your username and password to authenticate yourself in S³I. 

In [ ]:
print_with_timestamp("This is a KWH HMI, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your username and password are set.")

## Build and load the HMI JSON file 
Same as the digital twin, a HMI can also be created and launched using fml40 reference implementation library after the setting of the user credentials. All defined fml40 roles, functionalities and values etc. are respectively modelled according to the fml40 language. Next, you can create a JSON file which can be loaded to build a HMI reference.

In [ ]:
hmi_name = "my HMI"
config_file_name = make_thing_config(dt_id=hmi_id, name=hmi_name, roles=[{"class": "ml40::HMI"}])
hmi_model = load_config('configs/{}'.format(config_file_name))
hmi = create_dt_ref(model=hmi_model, grant_type="password", secret=hmi_secret,
                        username=username, password=password, 
                        is_broker=True, is_repo=False, is_broker_rest=True)

## Setup a looger for the HMI

In [ ]:
setup_logger(hmi_name)

## Run the HMI 
The created HMI is going to be launched in the following step. When the run_forever function is called, the HMI reference is started, and meanwhile its listener is also started to receive the messages sent to it via S³I Broker. Additionally, you can specific the receiver, with whom it will communicate. 

In [ ]:
hmi.run_forever()
receiver = input("[S³I]: Please enter the id of your digital twin: ")
hmi_endpoint = find_broker_endpoint(hmi.dir, hmi_id)
receiver_endpoint = find_broker_endpoint(hmi.dir, thing_id=receiver)

## Get the thing entry of the digital twin

In [ ]:
%%html
<img src="harvester_images/dir.png", width=500, height=500>

In [ ]:
print_with_timestamp("The thing entry of the digital twin: " + json.dumps(hmi.dir.queryThingIDBased(thingID=receiver), indent=2))

## Get the cloud copy of the digital twin 

In [ ]:
%%html
<img src="harvester_images/repo.png", width=500, height=500>

In [ ]:
print_with_timestamp("The cloud copy of the digital twin" + json.dumps(hmi.repo.queryThingIDBased(thingID=receiver), indent=2))

## Edit a felling job
we use this HMI in the next steps to create a felling job and then send it to the created digital twin.

In [ ]:
subFeatures = [{
            "class": "fml40::Assortment",
            "grade": "fl",
            "name": "Stammholz Abschnitte",
            "subFeatures": [
                {
                    "class": "fml40::ThicknessClass",
                    "name": ">"
                },
                {
                    "class": "fml40::WoodQuality",
                    "name": "B-C"
                },
                {
                    "class": "fml40::HarvestingParameters",
                    "cuttingLengths": 20
                },
                {
                    "class": "fml40::TreeType",
                    "name": "Spruce",
                    "conifer": True
                },
                {
                    "class": "fml40::HarvestedVolume",
                    "volume": 140
                }
            ]
        }]
feature_config_json = make_feature_config(class_name="fml40::FellingJob", subFeatures=subFeatures)
felling_job = build_feature(feature=feature_config_json)

print_with_timestamp("Felling job created:\n" + json.dumps(felling_job.to_json(), indent=2))

## Send a service request to your digital twin
In the follow steps we will show you how to edit a S3I-B ServiceRequest to call a service function in your digital twin. As prepared you can ask your dt to accept the felling job, query the current status of your felling job and remove a felling job from your digital twin. Just go to drop menu to run this block again to repeat sending service request again. 

In [ ]:
def make_ser_param_type():
    """
    This function is used to make the service parameters and type for a S³I-B service request
    """
    while True:
        service_type = input('[S3I]: What kind of service request would you like to choose? \n<fml40::AcceptsFellingJobs/acceptJob> \n<fml40::AcceptsFellingJobs/queryJobStatus> \n<fml40::AcceptsFellingJobs/removeJob> \n')
        if "fml40::AcceptsFellingJobs/acceptJob" in service_type: 
            parameters = {"job": felling_job.to_json()}
        elif "fml40::AcceptsFellingJobs/queryJobStatus" in service_type:
            job_id = input("[S³I]: Please enter the job id to query the job status: ")
            parameters={"identifier": job_id}
        elif "fml40::AcceptsFellingJobs/removeJob" in service_type:
            job_id = input("[S³I]: Please enter the job id to delete the job: ")
            parameters={"identifier": job_id}
        else:
            print_with_timestamp("[S³I]: Error in service type, please rewrite it!")
            parameters = None
        
        if parameters is None:
            continue
        else:
            return parameters, service_type
        
def prepare_service_req():
    serv_req = s3i.messages.ServiceRequest()
    parameters, service_type = make_ser_param_type()
    serv_req.fillServiceRequest(
        senderUUID=hmi_id, receiverUUID=[receiver], sender_endpoint=hmi_endpoint,
        serviceType=service_type,
        parameters=parameters,
        msgUUID="s3i:{}".format(uuid.uuid4())
    )
    return serv_req.msg

resp = hmi.broker.send([receiver_endpoint], json.dumps(prepare_service_req()))
while True: 
    send_serv_again = input("[S3I]: Do you want to send a service request again(j/n): ")
    if send_serv_again in yes:
        resp = hmi.broker.send([receiver_endpoint], json.dumps(prepare_service_req()))
    elif send_serv_again in no:
        break

## Send a get value request to your digital twin
In addition, the created digital twin is supposed to handle a get value request. As below we prepare and send a get value request to the digital twin and wait for the response. 

In [ ]:
getv_req = s3i.GetValueRequest()
path = input("[S³I]: Please enter the attribute path that you want to query: \ne.g. [] \n[attributes/features/ml40::Composite/targets/ml40::Engine/features/ml40::RotationalSpeed/rpm] ")
getv_req.fillGetValueRequest(
    senderUUID=hmi_id, receiverUUID=[receiver], sender_endpoint=hmi_endpoint,
    attributePath=path, msgUUID="s3i:{}".format(uuid.uuid4())

)
receiver_endpoint = find_broker_endpoint(hmi.dir, thing_id=receiver)
resp = hmi.broker.send([receiver_endpoint], json.dumps(getv_req.msg))